# 부동산 허위매물 분류 해커톤

## Data Import


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

plt.style.use('bmh')
plt.rcParams['font.family'] = 'Malgun Gothic'

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/dacon/fakerealestate/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/dacon/fakerealestate/test.csv')
submit = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/dacon/fakerealestate/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes,columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df) *100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    
    return summ
summary(train)

In [ ]:
for i in [train,test]:
    i['제공플랫폼'] = i['제공플랫폼'].apply(lambda x: x.replace('플랫폼',''))
    i['게재일'] = pd.to_datetime(i['게재일'],format='%Y-%m-%d')
    i['년'] = i['게재일'].dt.year
    i['월'] = i['게재일'].dt.month
    i['일'] = i['게재일'].dt.day
    i['요일'] = i['게재일'].dt.weekday
del train['게재일'],test['게재일']

## EDA

- 결측치 : 전용면적(연속형),해당층,총층,방수,욕실수,총주차대수
- 총층와 해당층 관련된 유사관계
- 관리비: 0의 의미

In [ ]:
df_tr = train.copy()
df_te = test.copy()

df_tr = df_tr.drop('ID',axis=1)
df_te = df_te.drop('ID',axis=1)

In [ ]:
con_col = ['보증금','월세','전용면적','관리비','해당층','총층','방수','욕실수','총주차대수']
cat_col = [col for col in df_tr.columns if col not in con_col]
cat_col.remove('중개사무소')
cat_col.remove('허위매물여부')

### Total Distribution

In [ ]:
con_col

In [ ]:
fig,ax = plt.subplots(3,3,figsize=(10,8))
ax = ax.flatten()

for i,col in enumerate(con_col):
    sns.histplot(df_tr[col],ax=ax[i])

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(12,10))
ax = ax.flatten()

for i,col in enumerate(cat_col):
    sns.histplot(df_tr[col],ax=ax[i])

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12,8))
ax = ax.flatten()

for i, col in enumerate(con_col):
    sns.kdeplot(data=df_tr, x=col,ax=ax[i],color='blue')
    sns.kdeplot(data=df_te,x=col,ax=ax[i],color='red')
    ax[i].set_title(f'{col} KDE',weight='bold')


In [ ]:
fig,ax = plt.subplots(3,3,figsize=(11,8))
ax = ax.flatten()

for i,col in enumerate(con_col):
    sns.kdeplot(x=df_tr[col],hue=df_tr['허위매물여부'],ax=ax[i])

In [ ]:
fig,ax = plt.subplots(3,3,figsize=(11,8))
ax = ax.flatten()

for i,col in enumerate(con_col):
    sns.histplot(x=df_tr[col],hue=df_tr['허위매물여부'],ax=ax[i],multiple='dodge')

### 1.매물 확인방식

In [ ]:
sns.histplot(x=df_tr['매물확인방식'],hue=df_tr['허위매물여부'],multiple='dodge')

### 2. 보증금 & 월세
- 허위매물 보증금/월세 분포도 전체 보증금/월세를 따라감 => 별다른 전처리X
- 월세와 보증금 간의 관계: 일정부분의 상관관계?
- 월세가 0인 데이터도 존재 => 이는 모두 허위매물임

In [ ]:
df_tr.loc[df_tr['월세']<10000]

In [ ]:
sns.histplot(x=df_tr['보증금'].loc[df_tr['허위매물여부']==1])

In [ ]:
sns.regplot(x=df_tr['월세'],y=df_tr['관리비'],line_kws={'color':'red'})

In [ ]:
sns.regplot(x=df_tr['월세'],y=df_tr['보증금'],line_kws={'color':'red'})

### 3.전용면적
- 결측치가 존재 1/6정도(787개)
- 전용면적은 방의 갯수,월세,보증금 등 여럿 변수에 영향을 받는다.
- 최소면적인 17.5의 방이 300개
    - 당연하게도? 17.5인 방에 허위매물이 많음(**12%**)
    
- 월세 / 보증금과 그렇게 특이한 부분이 띄진 않음.
- 결측치 / 결측치가 아닌 data의 target 비율이 비슷함.


In [ ]:
sns.displot(x=df_tr['전용면적'].loc[df_tr['전용면적']==17.5],hue=df_tr['허위매물여부'],multiple='dodge')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,5))
sns.histplot(x=df_tr['전용면적'].isna(),hue=df_tr['허위매물여부'],ax=ax[0],stat='percent',multiple='dodge')
sns.histplot(df_tr['전용면적'].loc[df_tr['허위매물여부']==1], color='red', stat='percent',ax=ax[1])
ax[0].set_title('결측치에 따른 허위매물 분포',weight='bold')
ax[1].set_title('허위매물의 전용면적 분포',weight='bold')

In [ ]:
df_tr.loc[df_tr['전용면적'].isna()]

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(11,7))
ax=ax.flatten()
sns.regplot(x='전용면적',y='보증금',data=df_tr,line_kws={'color':'black'},ax=ax[0])
sns.regplot(x='전용면적',y='월세',data=df_tr,line_kws={'color':'black'},ax=ax[1])
sns.scatterplot(x='전용면적',y='보증금',data=df_tr,hue='허위매물여부',ax=ax[2],alpha=0.7)
sns.scatterplot(x='전용면적',y='월세',data=df_tr,hue='허위매물여부',ax=ax[3],alpha=0.7)

In [ ]:
sns.scatterplot(x=df_tr['총주차대수'],y=df_tr['전용면적'],hue=df_tr['방수'],alpha=0.6)

### 4. 해당층 / 총층
- 해당층 // 총층의 값을 '층비율' 변수 정의로 하면 좋을듯.

- 총층이 결측치면 무조건 해당층도 결측치임. (총층:16개 / 해당층:229)
    - 총층 결측치는 삭제해도 괜찮을듯
    - 결측치 내부의 허위매물 비율도 별 특징 없는듯


In [ ]:
df_tr.loc[df_tr['총층'].isna()]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 5))
sns.histplot(x=df_tr['해당층'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[0])
ax[0].set_title('해당층 분포',weight='bold')

sns.histplot(x=df_tr['총층'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[1])
ax[1].set_title('총층 분포',weight='bold')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sns.countplot(x=df_tr['해당층'].isna(), hue=df_tr['허위매물여부'], ax=ax[0])
ax[0].set_title('해당층 결측치에 따른 허위매물 분포', weight='bold')

sns.countplot(x=df_tr['총층'].isna(), hue=df_tr['허위매물여부'], ax=ax[1])
ax[1].set_title('총층 결측치에 따른 허위매물 분포', weight='bold')
# Adding percentage labels on the bars
for a in ax:
    for p in a.patches:
        height = p.get_height()
        total = len(df_tr)
        a.text(p.get_x() + p.get_width() / 2., height + 3,
               f'{height/total:.1%}', ha="center")

In [ ]:
sns.scatterplot(data=df_tr, x='총층', y='해당층', alpha=0.5)

### 5. 방향

In [ ]:
sns.histplot(x=df_tr['방향'],hue=df_tr['허위매물여부'],multiple='dodge',stat='percent')

### 6. 방수 / 욕실수
- 결측치가 별로 없음. (방수:16 / 욕실:18)
    - 방수,총층의 결측치(16개)가 모두 겹침(이부분은 제거하는게 맞을듯)
    - 결측치의 합집합은 1171 칼럼.
- 방 수에 따라서 허위매물에 영향을 끼치는게 있을듯. (특히 2개)

In [ ]:
df_tr[['전용면적','해당층','총층','방수','욕실수','총주차대수']][df_tr.isna().any(axis=1)]

In [ ]:
df_tr.loc[df_tr['방수'].isna() & df_tr['총층'].isna()]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.countplot(x=df_tr['방수'], hue=df_tr['허위매물여부'], ax=ax[0])
ax[0].set_title('방수에 따른 허위매물 분포', weight='bold')

sns.countplot(x=df_tr['욕실수'], hue=df_tr['허위매물여부'], ax=ax[1])
ax[1].set_title('욕실수에 따른 허위매물 분포', weight='bold')

# Adding percentage labels on the bars
for p in ax[0].patches:
    height = p.get_height()
    total = len(df_tr)
    ax[0].text(p.get_x() + p.get_width() / 2., height + 3,
               f'{height/total:.1%}', ha="center")

for p in ax[1].patches:
    height = p.get_height()
    total = len(df_tr)
    ax[1].text(p.get_x() + p.get_width() / 2., height + 3,
               f'{height/total:.1%}', ha="center")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sns.countplot(x=df_tr['방수'].isna(), hue=df_tr['허위매물여부'], ax=ax[0])
ax[0].set_title('방수 결측치에 따른 허위매물 분포', weight='bold')

sns.countplot(x=df_tr['욕실수'].isna(), hue=df_tr['허위매물여부'], ax=ax[1])
ax[1].set_title('욕실수 결측치에 따른 허위매물 분포', weight='bold')
# Adding percentage labels on the bars
for a in ax:
    for p in a.patches:
        height = p.get_height()
        total = len(df_tr)
        a.text(p.get_x() + p.get_width() / 2., height + 3,
               f'{height/total:.1%}', ha="center")

### 7.주차가능여부 / 총주차대수
- 총주차대수의 결측치에 꽤 많은 허위매물이 존재함.
- 100~300대의 주차공간이 가능한 매물 / 600개 이상의 주차공간을 가진 곳도 존재
    - 이상치로 치부할것인가? => **이상치 고려X**

- 주차가능여부가 불가능인데 총주차대수가 기록되어있는 데이터??
    - 총 데이터는 630개
    - 하지만 결측치가 있는 데이터 696개
    

In [ ]:
df_tr.loc[df_tr['총주차대수'].isna()][['주차가능여부','총주차대수']]

In [ ]:
df_tr.loc[df_tr['총주차대수'].isna()]['주차가능여부'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(x=df_tr['주차가능여부'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[0])
ax[0].set_title('주차가능여부에 따른 허위매물 분포', weight='bold')
 
sns.histplot(x=df_tr['총주차대수'], hue=df_tr['허위매물여부'], ax=ax[1],multiple='dodge')
ax[1].set_title('총주차대수에 따른 허위매물 분포', weight='bold')
ax[1].set_xlim(0,300)
# Adding percentage labels on the bars
for a in ax:
    for p in a.patches:
        height = p.get_height()
        total = len(df_tr)
        if height / total > 0.0:
            a.text(p.get_x() + p.get_width() / 2., height + 3,
               f'{height/total:.1%}', ha="center")
plt.tight_layout()

In [ ]:
df_miss = df_tr.loc[(df_tr['주차가능여부']=='불가능')][['허위매물여부','총주차대수']]
df_miss.fillna(-1, inplace=True)
fig, ax = plt.subplots(figsize=(15, 5))
sns.countplot(x=df_miss['총주차대수'].loc[df_miss['총주차대수']<50], hue=df_miss['허위매물여부'], ax=ax)
# Adding percentage labels on the bars
total = len(df_miss)
for p in ax.patches:
    height = p.get_height()
    if height / total > 0.0:
        ax.text(p.get_x() + p.get_width() / 2., height + 3,
                f'{height/total:.1%}', ha="center")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(x=df_tr['총주차대수'].isna(), hue=df_tr['허위매물여부'], multiple='dodge', stat='percent', ax=ax[0])
ax[0].set_title('총주차대수 결측치에 따른 허위매물 분포', weight='bold')
sns.histplot(x=df_tr['총주차대수'].loc[df_tr['허위매물여부']==1], ax=ax[1],color='red')
ax[1].set_title('허위매물의 총주차대수 분포', weight='bold')

### 8. 관리비
- 관리비에 이상치가 몇개 존재하는듯
    
- 관리비가 0인 데이터가 많음 => 0의 의미?

In [ ]:
print(df_tr['관리비'].loc[df_tr['허위매물여부']==1].value_counts(normalize=True)[:5])
df_tr.loc[df_tr['관리비']>20].sort_values('관리비',ascending=False)

In [ ]:
#0인 허위매물의 비율이 30%
sns.displot(x=df_tr['관리비'].loc[df_tr['관리비']<20],hue=df_tr['허위매물여부'],multiple='dodge')

### 9. 중개사무소 / 제공플랫폼
- 중개사무소에서 사기가 시작될 확률이 높다.
    - 특정 중개사무소에 꽤 사건이 발생했음.
    - 총 132개의 중개사무소에서 허위매물이 발생했음


In [ ]:
df_tr['제공플랫폼'].value_counts()

In [ ]:
#허위매물이 있는 중개사무소 이름 (허위매물이 많은 순서대로)
estate_agency = df_tr.loc[df_tr['허위매물여부']==1].value_counts(subset=['중개사무소']).to_frame().reset_index()['중개사무소'].to_list()

In [ ]:
df_tr.loc[(df_tr['중개사무소'].isin(estate_agency[:2]))&(df_tr['허위매물여부']==1)]

In [ ]:
df_tr.loc[(df_tr['중개사무소']=='G52Iz8V2B9') & (df_tr['허위매물여부']==1)]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.histplot(x=df_tr['제공플랫폼'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax)

# Adding percentage labels on the bars
total = len(df_tr)
for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.text(p.get_x() + p.get_width() / 2., height + 3,
                f'{height/total:.1%}', ha="center")

### 10. 년/월/일/요일
- 요일에 따라서는 진짜 구분이 없음. => 요일 feature 삭제 고려

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(11, 7))
sns.histplot(df_tr['년'], ax=ax[0, 0])
ax[0, 0].set_title('년 분포', weight='bold')

sns.histplot(df_tr['월'], ax=ax[0, 1])
ax[0, 1].set_title('월 분포', weight='bold')

sns.histplot(df_tr['일'], ax=ax[1, 0])
ax[1, 0].set_title('일 분포', weight='bold')

sns.histplot(df_tr['요일'], ax=ax[1, 1])
ax[1, 1].set_title('요일 분포', weight='bold')

In [ ]:
df_tr.loc[(df_tr['허위매물여부']==1)&(df_tr['월']==4)]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(11, 7))

sns.histplot(x=df_tr['년'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[0, 0])
ax[0, 0].set_title('년 분포', weight='bold')

sns.histplot(x=df_tr['월'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[0, 1])
ax[0, 1].set_title('월 분포', weight='bold')

sns.histplot(x=df_tr['일'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[1, 0])
ax[1, 0].set_title('일 분포', weight='bold')

sns.histplot(x=df_tr['요일'], hue=df_tr['허위매물여부'], multiple='dodge', ax=ax[1, 1])
ax[1, 1].set_title('요일 분포', weight='bold')


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_tr.corr(numeric_only=True),annot=True)

## Feature Engineering
- <조건1> : 층비율, 결측치 16개 삭제, 요일삭제, 전용면적 평균 대체
- KNN Imputer 이용한 결측치 보간

In [ ]:
#층비율 파생변수 생성
df_tr['층비율'] = df_tr['해당층'] / df_tr['총층']
df_te['층비율'] = df_te['해당층'] / df_te['총층']
print(df_tr.shape)

In [ ]:
#전용면적 결측치 보간
df_tr['전용면적'] = df_tr['전용면적'].fillna(df_tr['전용면적'].mean())
df_tr['전용면적'].isna().sum()

In [ ]:
#결측치 16개 삭제
df_tr = df_tr.dropna(subset=['방수'],axis=0)
print(df_tr.shape)

In [ ]:
df_tr = df_tr.drop(columns=['요일'],axis=1)
df_te = df_te.drop(columns=['요일'],axis=1)
print(df_tr.shape)

cat_col.remove('요일')
cat_col.append('중개사무소')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_tr[cat_col] = df_tr[cat_col].apply(lambda x:le.fit_transform(x))
df_te[cat_col] = df_te[cat_col].apply(lambda x:le.fit_transform(x))

from sklearn.preprocessing import MinMaxScaler
mn = MinMaxScaler()
for col in [['보증금','월세']]:
    df_tr[col] = mn.fit_transform(df_tr[col])
    df_te[col] = mn.fit_transform(df_te[col])

df_tr.head()

In [ ]:
plt.figure(figsize=(11,11))
sns.heatmap(df_tr.corr(),annot=True)

In [ ]:
print(df_tr.shape,df_te.shape)
X,y = df_tr.drop('허위매물여부',axis=1),df_tr['허위매물여부']
X_test = df_te

## 모델 설계

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from optuna import Trial,visualization
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score
import optuna 

from sklearn.model_selection import StratifiedKFold

### RandomForest
- baseline : 0.72
- 요일 삭제 : 0.76
- 요일 삭제+dropna16 : 0.743
- 요일삭제+dropna16+전용면적보간 : 0.73

In [ ]:
rf_val_preds = np.zeros(X.shape[0])
rf_test_preds = np.zeros(X_test.shape[0])

folds = StratifiedKFold(n_splits=5,shuffle=True,random_state=2025)
f1_rf = []

for idx,(train_idx,val_idx) in enumerate(folds.split(X,y)):
    print(f'Fold {idx+1} / Fold {folds.n_splits}')
    X_train,y_train = X.iloc[train_idx],y.iloc[train_idx]
    X_val,y_val = X.iloc[val_idx],y.iloc[val_idx]

    rf = RandomForestClassifier(random_state=2025)
    rf.fit(X_train,y_train)
    rf_val_preds[val_idx] += rf.predict(X_val)
    
    
    print(f'Train f1_score: {f1_score(y_train,rf.predict(X_train))}')
    print(f'Valid f1_score: {f1_score(y_val,rf.predict(X_val))}')
    print('-'*30)

    rf_test_preds += rf.predict(X_test)/folds.n_splits
    
    f1_rf.append(f1_score(y_val,rf_val_preds[val_idx]))

print(f'F1-Score: {np.array(f1_rf).mean()}')

In [ ]:
sns.barplot(pd.Series(rf.feature_importances_,index=X_test.columns),orient='h')

### LightGBM
- 결측치 (16개) 삭제 : 0.826 (실제로는 낮게 측정되어 나옴)
- 0.819 : 요일 삭제
- 요일+결측치+층비율 : 0.816 <조건2>
- 요일+결측치(16,전용면적)+층비율 : 0.818 <조건1> (public best)
- ~~조건1+년도 삭제 : 0.758~~
- ~~총주차대수 결측치 0 대입: 최악~~
- 조건1+관리비 이상치 삭제: 0.829 (실제로 낮음)
- 조건1+'일' 삭제: 0.821
- ~~조건1에 minmaxscaler : 0.823~~
- 조건1+해당층,총층 삭제: 0.822
- 조건1 + 총주차대수 결측치 평균으로 대체: 0.820
- 조건2 + 전용면적(방수 참고해서 fillna) : 0.819

#### baseline
- 0.845 : 말그대로 아무것도X
- 0.823 : 게재일 삭제, 년/월/일/요일 파생변수 생성


In [ ]:
lgb_val_preds = np.zeros(X.shape[0])
lgb_test_preds = np.zeros(X_test.shape[0])
lgb_val_pred_proba = np.zeros(X.shape[0])

folds = StratifiedKFold(n_splits=5,shuffle=True,random_state=2025)
f1_lgb = []
evals_result_all = []

for idx,(train_idx,val_idx) in enumerate(folds.split(X,y)):
    print(f'Fold {idx+1} / Fold {folds.n_splits}')
    X_train,y_train = X.iloc[train_idx],y.iloc[train_idx]
    X_val,y_val = X.iloc[val_idx],y.iloc[val_idx]

    lgb = LGBMClassifier(random_state=2025, objective='binary',
                         n_estimators=70,learning_rate=0.22)
    lgb.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)])
    lgb_val_preds[val_idx] = lgb.predict(X_val)
    lgb_val_pred_proba[val_idx] = lgb.predict_proba(X_val)[:,1]
    
    print(f'Train f1_score: {f1_score(y_train,lgb.predict(X_train))}')
    print(f'Valid f1_score: {f1_score(y_val,lgb.predict(X_val))}')
    print('-'*30)

    lgb_test_preds += lgb.predict(X_test)/folds.n_splits
    f1_lgb.append(f1_score(y_val,lgb_val_preds[val_idx]))

    evals_result_all.append(lgb.evals_result_)

print(f'F1-Score: {np.array(f1_lgb).mean()}')

In [ ]:
from lightgbm import plot_importance
plot_importance(lgb)

In [ ]:
# Plot the learning curves
fig, axes = plt.subplots(1, 5, figsize=(30, 5), sharey=True)
for i, evals_result in enumerate(evals_result_all):
    train_logloss = evals_result['training']['binary_logloss']
    valid_logloss = evals_result['valid_1']['binary_logloss']
    axes[i].plot(train_logloss, label='Training Log Loss')
    axes[i].plot(valid_logloss, label='Validation Log Loss')
    axes[i].set_title(f'Fold {i+1}')
    axes[i].set_xlabel('Iterations')
    axes[i].set_ylabel('Log Loss')
    axes[i].legend()


In [ ]:
#예측을 잘 못하는 데이터 값 추출
pred_proba_lgb_list = abs(y_val - lgb_val_pred_proba[val_idx]).sort_values(ascending=False)
X_val.loc[pred_proba_lgb_list.index][:20]

#### optuna

In [ ]:
def objectivelgb(trial:Trial,X,y):
    from sklearn.model_selection import train_test_split
    X_train_op,y_train_op,X_val_op,y_val_op = train_test_split(X,y,test_size=0.2,random_state=2025)
    
    global lgb_param
    lgb_param = {'random_state':2025,
                 'objective':'binary'}
    model = LGBMClassifier(**lgb_param)
    model.fit(X_train_op,y_train_op,eval_set=[(X_val_op,y_val_op)])
    
    answer = model.predict(X_val_op)
    score = f1_score(answer,y_val_op)
    
    return score

In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler(seed=2025))
study.optimize(lambda trial: objectivelgb(trial,X,y),n_trials=20)
print('Best trial: f1-score {},\nparams : {}'.format(study.best_trial.value,study.best_trial.params))

## 결과물 제출

In [ ]:
submit['허위매물여부'] = lgb_test_preds
submit['허위매물여부'] = submit['허위매물여부'].apply(lambda x: 1 if x > 0.5 else 0)
submit.head()

In [ ]:
submit.to_csv('lgb_term1_hyperparam2.csv',index=False)